<a href="https://colab.research.google.com/github/bengHak/Colab_Tensorboard/blob/master/tensorboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

커서를 첫 코드 셀에 두고  
Ctrl(Cmd_맥) + F9 ( 모두실행 ) 누르면 자동으로 전부 실행해준다.  
실행 전 모든 런타임 재설정 확인하면 됨.



로그파일과 텐서보드를 초기화한다.

In [0]:
!ls
!ls tb_logs/

!rm -rf ngrok
!rm ngrok-stable-linux-amd64.zip.1
!rm ngrok-stable-linux-amd64.zip
!rm -rf tb_logs/*
!ls tb_logs

raw.githubusercontent.com  sample_data	tb_logs
rm: cannot remove 'ngrok-stable-linux-amd64.zip.1': No such file or directory
rm: cannot remove 'ngrok-stable-linux-amd64.zip': No such file or directory


텐서보드 프로그램을 설치하고 로그를 가져오는 디렉토리를 설정한다.

In [0]:
# You can change the directory name

LOG_DIR = 'tb_logs'



!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

!unzip ngrok-stable-linux-amd64.zip



import os

if not os.path.exists(LOG_DIR):
    os.makedirs(LOG_DIR)
    get_ipython().system_raw('tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'.format(LOG_DIR))
    
get_ipython().system_raw('./ngrok http 6006 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
        "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"



--2019-05-23 14:23:40--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.200.123.104, 52.4.75.11, 52.201.75.180, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.200.123.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16648024 (16M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  15.88M  18.6MB/s    in 0.9s    

2019-05-23 14:23:41 (18.6 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [16648024/16648024]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
https://898c0ebd.ngrok.io


W, b, cost, accuracy 노드의 값들을 텐서보드에서 모니터링 할 수 있게  
tf.summary.merge_all()을 통해 텐서보드에 전달해준다.

In [0]:
# Lab 6 Softmax Classifier
import tensorflow as tf
import numpy as np
import pandas as pd

tf.set_random_seed(777)  # for reproducibility

# Predicting animal type based on various features

url = 'https://raw.githubusercontent.com/hunkim/DeepLearningZeroToAll/master/data-04-zoo.csv'
xy = np.loadtxt(url, delimiter=',', dtype=np.float32)
#xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype=np.float32)
#xy = pd.read_csv(url)
print(xy)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

print(x_data.shape, y_data.shape)

'''
(101, 16) (101, 1)
'''

nb_classes = 7  # 0 ~ 6

X = tf.placeholder(tf.float32, [None, 16])
Y = tf.placeholder(tf.int32, [None, 1])  # 0 ~ 6

Y_one_hot = tf.one_hot(Y, nb_classes)  # one hot
print("one_hot:", Y_one_hot)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])
print("reshape one_hot:", Y_one_hot)

'''
one_hot: Tensor("one_hot:0", shape=(?, 1, 7), dtype=float32)
reshape one_hot: Tensor("Reshape:0", shape=(?, 7), dtype=float32)
'''

W = tf.Variable(tf.random_normal([16, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)

# Cross entropy cost/loss
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,
                                                                 labels=tf.stop_gradient([Y_one_hot])))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

prediction = tf.argmax(hypothesis, 1)
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# TensorBoard
w_hist = tf.summary.histogram("weight", W) 
b_hist = tf.summary.histogram("bias", b) 
hypothesis_hist = tf.summary.histogram("y", hypothesis) 
cost_hist = tf.summary.scalar('cost_scalar', cost)
accuracy_hist = tf.summary.scalar('accuracy_scalar', accuracy)

merged = tf.summary.merge_all()

# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter('tb_logs', sess.graph)

    for step in range(2001):
        #_, cost_val, acc_val = sess.run([optimizer, cost, accuracy], feed_dict={X: x_data, Y: y_data})
        summary = sess.run(merged, feed_dict={X: x_data, Y: y_data})
        writer.add_summary(summary, step)
        writer.flush()
                                        
        if step % 100 == 0:
            print(summary)
            #print("Step: {:5}\tCost: {:.3f}\tAcc: {:.2%} \n".format(step, cost_val, acc_val))

    # Let's see if we can predict
    pred = sess.run(prediction, feed_dict={X: x_data})
    # y_data: (N,1) = flatten => (N, ) matches pred.shape
    for p, y in zip(pred, y_data.flatten()):
        print("[{}] Prediction: {} True Y: {} \n".format(p == int(y), p, int(y)))

'''
Step:     0 Loss: 5.106 Acc: 37.62%
Step:   100 Loss: 0.800 Acc: 79.21%
Step:   200 Loss: 0.486 Acc: 88.12%
...
Step:  1800	Loss: 0.060	Acc: 100.00%
Step:  1900	Loss: 0.057	Acc: 100.00%
Step:  2000	Loss: 0.054	Acc: 100.00%
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
...
[True] Prediction: 0 True Y: 0
[True] Prediction: 6 True Y: 6
[True] Prediction: 1 True Y: 1
'''

[[1. 0. 0. ... 0. 1. 0.]
 [1. 0. 0. ... 0. 1. 0.]
 [0. 0. 1. ... 0. 0. 3.]
 ...
 [1. 0. 0. ... 0. 1. 0.]
 [0. 0. 1. ... 0. 0. 6.]
 [0. 1. 1. ... 0. 0. 1.]]
(101, 16) (101, 1)
one_hot: Tensor("one_hot:0", shape=(?, 1, 7), dtype=float32)
reshape one_hot: Tensor("Reshape:0", shape=(?, 7), dtype=float32)
Instructions for updating:
Colocations handled automatically by placer.
b'\n\xd0\t\n\x08weight_1*\xc3\t\t\x00\x00\x00`\xe6+\x05\xc0\x11\x00\x00\x00\x80g\xe2\x02@\x19\x00\x00\x00\x00\x00\x00\\@!\x00\x00\xe0Z\xc3\x01%\xc0)\xc8\xcc5\xb5A\x1e_@2\xc8\x04h\xbc\x105\x9a\x07\x07\xc0\xea6\x0fv\xa3\xef\x04\xc0w`<\x0ef\x08\x03\xc0\xe0\x11\xf1\x0ctM\x01\xc0\xdc\x94\xcd.\x8du\xff\xbfS\x87F\x13i\x99\xfc\xbfyL\xfa\xe2\xd3\xff\xf9\xbf\x85E\xcc\x9f\xc0\xa2\xf7\xbf3?\xe8\x1c\xaf|\xf5\xbf\xbaP\x8d1\x9f\x88\xf3\xbfcI\xc6\x15\x05\xc2\xf1\xbf+\x14\xfa\xcd\xbe$\xf0\xbf\x08\x99i\x19\x15Z\xed\xbf\x1e\x8b\x8e\xe8\xfb\xae\xea\xbf2g\xc7G\xfcA\xe8\xbf\x16uo\xfbp\r\xe6\xbf+Se*8\x0c\xe4\xbf\x0f\x1d\\l\xa79\xe2\xbf\xb01%

'\nStep:     0 Loss: 5.106 Acc: 37.62%\nStep:   100 Loss: 0.800 Acc: 79.21%\nStep:   200 Loss: 0.486 Acc: 88.12%\n...\nStep:  1800\tLoss: 0.060\tAcc: 100.00%\nStep:  1900\tLoss: 0.057\tAcc: 100.00%\nStep:  2000\tLoss: 0.054\tAcc: 100.00%\n[True] Prediction: 0 True Y: 0\n[True] Prediction: 0 True Y: 0\n[True] Prediction: 3 True Y: 3\n...\n[True] Prediction: 0 True Y: 0\n[True] Prediction: 6 True Y: 6\n[True] Prediction: 1 True Y: 1\n'